![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [19]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso  # Added Lasso import here
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Import any additional modules and start coding below
rental_info = pd.read_csv('rental_info.csv')
rental_info.head(5)
returns = pd.to_datetime(rental_info["return_date"])  # Fixed the syntax error here
rentals = pd.to_datetime(rental_info["rental_date"])
diff = returns - rentals
rental_info["rental_length_days"] = diff.dt.days  # Corrected to use .dt.days for Series
rental_info.head(5)

rental_info["deleted_scenes"] = np.where(rental_info["special_features"].str.contains("Deleted Scenes"), 1,0)
rental_info["behind_the_scenes"] = np.where(rental_info["special_features"].str.contains("Behind the Scenes"), 1,0)
rental_info.head(5)

# The feature names were correctly listed, but we need to select these features from the DataFrame for X, and select the target variable for y.
X_features = ['amount', 'release_year', 'rental_rate', 'length', 'replacement_cost', 'NC-17', 'PG', 'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2', 'deleted_scenes', 'behind_the_scenes']
X = rental_info[X_features]  # Selecting the actual data for features from the DataFrame
y = rental_info['rental_length_days']  # Selecting the target variable from the DataFrame

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

lasso = Lasso(alpha = 1.0)
# fit model
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_
print('Lasso coefficiets: ', lasso_coef)

# Perform feature selection by choosing columns with positive coefficients
X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

# Run OLS models on lasso chosen regression
ols = LinearRegression()
ols = ols.fit(X_lasso_train, y_train)
y_test_pred = ols.predict(X_lasso_test)
mse_lin_reg_lasso = mean_squared_error(y_test, y_test_pred)
print('LinearRegression(Lasso) MSE: ', mse_lin_reg_lasso)

# Random forest hyperparameter space
param_dist = {'n_estimators': np.arange(1,101,1),
          'max_depth':np.arange(1,11,1)}

# Create a random forest regressor
rf = RandomForestRegressor()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions=param_dist, 
                                 cv=5, 
                                 random_state=9)

# Fit the random search object to the data
rand_search.fit(X_train, y_train)

# Create a variable for the best hyper param
hyper_params = rand_search.best_params_

# Run the random forest on the chosen hyper parameters
rf = RandomForestRegressor(n_estimators=hyper_params["n_estimators"], 
                           max_depth=hyper_params["max_depth"], 
                           random_state=9)
rf.fit(X_train,y_train)
rf_pred = rf.predict(X_test)
mse_random_forest= mean_squared_error(y_test, rf_pred)

# Random forest gives lowest MSE so:
best_model = rf
print('Best model: ', best_model)  # Corrected to print the variable best_model instead of getbest_model
best_mse = mse_random_forest
print('Best MSE: ', best_mse)
#make a prediction
y_pred = best_model.predict(X_test)  # Corrected to use best_model for prediction instead of model

Lasso coefficiets:  [ 0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  8.89702167e-02  1.99142245e-06 -1.16107154e-01
 -0.00000000e+00  0.00000000e+00]
LinearRegression(Lasso) MSE:  4.883429956984267
Best model:  RandomForestRegressor(max_depth=10, n_estimators=51, random_state=9)
Best MSE:  2.225667528098759
